In [16]:
from dowhy import CausalModel
import pandas as pd
import itertools
import warnings
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('./out/long_df.csv')
print(df.dtypes)

Unnamed: 0            int64
gameid               object
date                 object
league               object
patch               float64
playoffs              int64
gamelength            int64
days_since_patch      int64
golddiffat15        float64
recent_patch          int64
teamid               object
result                int64
win_prob            float64
large_lead            int64
underdog               bool
elo                 float64
kills               float64
winstreak           float64
visionscore         float64
side                 object
side_adv              int64
dtype: object


In [6]:
#We want to iterate over all possible confounders and check for the causal effect
#First we use domain knowledge to pinpoint a list of posibble confounder/covariats
possible_confounder = ['playoffs', 'gamelength', 'win_prob', 'winstreak', 'side_adv']

In [12]:
def calculate_causal_effect(df, treatment, outcome, confounders):
    model = CausalModel(
        data=df,
        treatment=treatment,
        outcome=outcome,
        common_causes=confounders
    )
    
    identified_estimand = model.identify_effect()

    estimate = model.estimate_effect(
        identified_estimand,
        method_name='backdoor.linear_regression'
    )
    return estimate.value

In [15]:
#Now we iterate over possible combinations 
treatment = 'golddiffat15'
outcome = 'result'
effect_dict = {}

warnings.simplefilter(action='ignore', category=FutureWarning)

#Iterate over combinations of all lengths 
for r in range(1, len(possible_confounder)+1):
    for combo in itertools.combinations(possible_confounder, r):
        effect_dict[combo] = calculate_causal_effect(df, treatment, outcome, list(combo))

#Now find the max causal effect
max_effect_combo = max(effect_dict, key=effect_dict.get)
max_effect = effect_dict[max_effect_combo]

print(f'The maximum causal effect was witnessed with confounders: {max_effect_combo} and is {max_effect}')
avg_effect = sum(effect_dict.values()) / len(effect_dict)
print(f"Average causal effect: {avg_effect}")

The maximum causal effect was witnessed with confounders: ('playoffs',) and is 0.00013528277601465577
Average causal effect: 0.00013463844132677853
